In [72]:
import pandas as pd
import numpy as np
from apyori import apriori

In [54]:
df = pd.read_csv('/home/marcela/projetos/models_ml2/apriori/compras_mercado/mercado.csv', header=None)

In [55]:
df

,0,1,2,3
0,leite,cerveja,pão,manteiga
1,café,pão,manteiga,NaN
2,café,pão,manteiga,NaN
3,leite,café,pão,manteiga
4,cerveja,NaN,NaN,NaN
5,manteiga,NaN,NaN,NaN
6,pão,NaN,NaN,NaN
7,feijão,NaN,NaN,NaN
8,arroz,feijão,NaN,NaN
9,arroz,NaN,NaN,NaN


In [56]:
# Converting to list
df_list = []
number_registers = len(df)
for i in range(number_registers):
    register = list(df.iloc[i])
    df_list.append(register)
print(df_list)

[['leite', 'cerveja', 'pão', 'manteiga'], ['café', 'pão', 'manteiga', nan], ['café', 'pão', 'manteiga', nan], ['leite', 'café', 'pão', 'manteiga'], ['cerveja', nan, nan, nan], ['manteiga', nan, nan, nan], ['pão', nan, nan, nan], ['feijão', nan, nan, nan], ['arroz', 'feijão', nan, nan], ['arroz', nan, nan, nan]]


In [90]:
# Converting to list
df_list = []
number_registers = df.shape[0]
order_item = df.shape[1]
for i in range(number_registers):
    df_list.append([str(df.values[i,j]) for j in range(order_item)])
print(df_list)

[['leite', 'cerveja', 'pão', 'manteiga'], ['café', 'pão', 'manteiga', 'nan'], ['café', 'pão', 'manteiga', 'nan'], ['leite', 'café', 'pão', 'manteiga'], ['cerveja', 'nan', 'nan', 'nan'], ['manteiga', 'nan', 'nan', 'nan'], ['pão', 'nan', 'nan', 'nan'], ['feijão', 'nan', 'nan', 'nan'], ['arroz', 'feijão', 'nan', 'nan'], ['arroz', 'nan', 'nan', 'nan']]


#### Model

In [97]:
model = apriori(df_list, min_support=0.3, min_confidence=0.8, min_lift=2)

In [98]:
results = list(model)

In [99]:
results

[RelationRecord(items=frozenset({'manteiga', 'café'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'café'}), items_add=frozenset({'manteiga'}), confidence=1.0, lift=2.0)]),
 RelationRecord(items=frozenset({'pão', 'café'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'café'}), items_add=frozenset({'pão'}), confidence=1.0, lift=2.0)]),
 RelationRecord(items=frozenset({'pão', 'manteiga', 'café'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'café'}), items_add=frozenset({'pão', 'manteiga'}), confidence=1.0, lift=2.5), OrderedStatistic(items_base=frozenset({'manteiga', 'café'}), items_add=frozenset({'pão'}), confidence=1.0, lift=2.0), OrderedStatistic(items_base=frozenset({'pão', 'café'}), items_add=frozenset({'manteiga'}), confidence=1.0, lift=2.0)])]

In [101]:
rules = results[2][2]

In [102]:
rules

[OrderedStatistic(items_base=frozenset({'café'}), items_add=frozenset({'pão', 'manteiga'}), confidence=1.0, lift=2.5),
 OrderedStatistic(items_base=frozenset({'manteiga', 'café'}), items_add=frozenset({'pão'}), confidence=1.0, lift=2.0),
 OrderedStatistic(items_base=frozenset({'pão', 'café'}), items_add=frozenset({'manteiga'}), confidence=1.0, lift=2.0)]

In [110]:
A = []
B = []
support = []
confidence = []
lift = []

for result in results:
    s = result[1]
    result_rules = result[2]
    for result_rule in result_rules:
        a = list(result_rule[0])
        b = list(result_rule[1])
        c = result_rule[2]
        l = result_rule[3]

        A.append(a)
        B.append(b)
        support.append(s)
        confidence.append(c)
        lift.append(l)

In [115]:
df_rules = pd.DataFrame({'A': A, 'B': B, 'support': support, 'confidence': confidence, 'lift': lift})

In [116]:
df_rules

,A,B,support,confidence,lift
0,[café],[manteiga],0.3,1.0,2.0
1,[café],[pão],0.3,1.0,2.0
2,[café],"[pão, manteiga]",0.3,1.0,2.5
3,"[manteiga, café]",[pão],0.3,1.0,2.0
4,"[pão, café]",[manteiga],0.3,1.0,2.0


In [117]:
df_rules.sort_values(by='lift', ascending=False)

,A,B,support,confidence,lift
2,[café],"[pão, manteiga]",0.3,1.0,2.5
0,[café],[manteiga],0.3,1.0,2.0
1,[café],[pão],0.3,1.0,2.0
3,"[manteiga, café]",[pão],0.3,1.0,2.0
4,"[pão, café]",[manteiga],0.3,1.0,2.0


## ECLAT

In [120]:
from pyECLAT import ECLAT

In [121]:
eclat = ECLAT(data=df)

In [123]:
eclat.df_bin

,pão,manteiga,café,cerveja,leite,feijão,arroz
0,1,1,0,1,1,0,0
1,1,1,1,0,0,0,0
2,1,1,1,0,0,0,0
3,1,1,1,0,1,0,0
4,0,0,0,1,0,0,0
5,0,1,0,0,0,0,0
6,1,0,0,0,0,0,0
7,0,0,0,0,0,1,0
8,0,0,0,0,0,1,1
9,0,0,0,0,0,0,1


In [124]:
indexes, support_2 = eclat.fit(min_support=0.3, min_combination=1, max_combination=3)

Combination 1 by 1


3it [00:00, 44.15it/s]


Combination 2 by 2


3it [00:00, 59.47it/s]


Combination 3 by 3


1it [00:00, 61.62it/s]


In [125]:
indexes

{'pão': [0, 1, 2, 3, 6],
 'manteiga': [0, 1, 2, 3, 5],
 'café': [1, 2, 3],
 'pão & manteiga': [0, 1, 2, 3],
 'pão & café': [1, 2, 3],
 'manteiga & café': [1, 2, 3],
 'pão & manteiga & café': [1, 2, 3]}

In [126]:
support_2

{'pão': 0.5,
 'manteiga': 0.5,
 'café': 0.3,
 'pão & manteiga': 0.4,
 'pão & café': 0.3,
 'manteiga & café': 0.3,
 'pão & manteiga & café': 0.3}